In [ ]:
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
!unzip /content/dataset.zip

Archive:  /content/dataset.zip
   creating: dataset/test/
   creating: dataset/test/Compost/
  inflating: dataset/test/Compost/O_12787.jpg  
  inflating: dataset/test/Compost/O_12788.jpg  
  inflating: dataset/test/Compost/O_12789.jpg  
  inflating: dataset/test/Compost/O_12790.jpg  
  inflating: dataset/test/Compost/O_12791.jpg  
  inflating: dataset/test/Compost/O_12792.jpg  
  inflating: dataset/test/Compost/O_12793.jpg  
  inflating: dataset/test/Compost/O_12794.jpg  
  inflating: dataset/test/Compost/O_12795.jpg  
  inflating: dataset/test/Compost/O_12796.jpg  
  inflating: dataset/test/Compost/O_12797.jpg  
  inflating: dataset/test/Compost/O_12798.jpg  
  inflating: dataset/test/Compost/O_12799.jpg  
  inflating: dataset/test/Compost/O_12800.jpg  
  inflating: dataset/test/Compost/O_12801.jpg  
  inflating: dataset/test/Compost/O_12802.jpg  
  inflating: dataset/test/Compost/O_12803.jpg  
  inflating: dataset/test/Compost/O_12804.jpg  
  inflating: dataset/test/Compost/O_12805.j

In [ ]:
base_dir = 'dataset/train'

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset='training'
)
val_generator = datagen.flow_from_directory(  #validation generator
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 56 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [ ]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'Compost': 0, 'Recycling': 1}


In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [ ]:
base_model.trainable=False
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, #no.of classes
                        activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
epochs = 10
history = model.fit(
    train_generator, 
    epochs = epochs, 
    validation_data=val_generator
)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.9432 - accuracy: 0.4286 - val_loss: 3.0204 - val_accuracy: 0.8462
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 3.9152 - accuracy: 0.8036 - val_loss: 2.5866 - val_accuracy: 0.8462
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 2.6848 - accuracy: 0.8036 - val_loss: 1.6805 - val_accuracy: 0.8462
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 1.1505 - accuracy: 0.8036 - val_loss: 0.7799 - val_accuracy: 0.8462
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 0.2612 - accuracy: 0.8393 - val_loss: 0.2465 - val_accuracy: 0.9231
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 0.1152 - accuracy: 0.8750 - val_loss: 0.0967 - val_accuracy: 0.9231
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 0.1028 - accuracy: 0.9464 - val_loss: 0.0523 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=========================

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmp9fn1w_hi/assets


INFO:tensorflow:Assets written to: /tmp/tmp9fn1w_hi/assets


In [ ]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>